In [ ]:
# Setup and GPU Check
import torch
print("PyTorch version:", torch.__version__)
print("GPU available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Imports
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, models
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os
import time


In [ ]:
# Completed project in Google Colab with dataset loaded in Google Drive
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base_path = '/content/drive/MyDrive/archive (2)/BreaKHis_v1/BreaKHis_v1/histology_slides/breast'


In [ ]:
# Dataset Class
class BreakHisDataset(Dataset):
    """Custom dataset for BreakHis histopathology images"""
    def __init__(self, base_path, magnification='400X', transform=None):
        self.base_path = base_path
        self.magnification = magnification
        self.transform = transform
        self.images = []
        self.labels = []

        for class_name, label in [('benign', 0), ('malignant', 1)]:
            class_path = os.path.join(base_path, class_name)
            for root, dirs, files in os.walk(class_path):
                if magnification in root:
                    for file in files:
                        if file.endswith('.png'):
                            self.images.append(os.path.join(root, file))
                            self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]

In [ ]:
# Image Preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [ ]:
# Load All Magnifications
datasets_all_mags = {}

for mag in ['40X', '100X', '200X', '400X']:
    dataset = BreakHisDataset(base_path, magnification=mag, transform=transform)

    indices = list(range(len(dataset)))
    labels = dataset.labels

    train_idx, temp_idx = train_test_split(indices, test_size=0.30, random_state=42, stratify=labels)
    temp_labels = [labels[i] for i in temp_idx]
    val_idx, test_idx = train_test_split(temp_idx, test_size=0.50, random_state=42, stratify=temp_labels)

    train_dataset = Subset(dataset, train_idx)
    val_dataset = Subset(dataset, val_idx)
    test_dataset = Subset(dataset, test_idx)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    datasets_all_mags[mag] = {
        'dataset': dataset,
        'train_loader': train_loader,
        'val_loader': val_loader,
        'test_loader': test_loader
    }

In [ ]:
# Load ResNet50 Feature Extractor
resnet = models.resnet50(weights='IMAGENET1K_V1')
feature_extractor = nn.Sequential(*list(resnet.children())[:-1])
feature_extractor = feature_extractor.to(device)
feature_extractor.eval()

def extract_features(dataloader, extractor):
    features = []
    labels = []
    
    with torch.no_grad():
        for images, lbls in tqdm(dataloader):
            images = images.to(device)
            feats = extractor(images)
            feats = feats.view(feats.size(0), -1)
            features.append(feats.cpu().numpy())
            labels.append(lbls.numpy())
    
    return np.vstack(features), np.concatenate(labels)


In [ ]:
# Extract Features for All Magnifications
all_features = {}

for mag in ['40X', '100X', '200X', '400X']:
    train_loader = datasets_all_mags[mag]['train_loader']
    val_loader = datasets_all_mags[mag]['val_loader']
    test_loader = datasets_all_mags[mag]['test_loader']

    X_train, y_train = extract_features(train_loader, feature_extractor)
    X_val, y_val = extract_features(val_loader, feature_extractor)
    X_test, y_test = extract_features(test_loader, feature_extractor)

    all_features[mag] = {
        'X_train': X_train, 'y_train': y_train,
        'X_val': X_val, 'y_val': y_val,
        'X_test': X_test, 'y_test': y_test
    }

In [ ]:
# Define Traditional ML Models
f1_scorer = make_scorer(f1_score)

models_params = {
    'Logistic Regression': {
        'model': LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42),
        'params': {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l2']}
    },
    'Linear SVM': {
        'model': SVC(kernel='linear', class_weight='balanced', random_state=42),
        'params': {'C': [0.01, 0.1, 1, 10, 100]}
    },
    'RBF SVM': {
        'model': SVC(kernel='rbf', class_weight='balanced', random_state=42),
        'params': {'C': [0.1, 1, 10], 'gamma': ['scale', 0.001, 0.01, 0.1]}
    },
    'Random Forest': {
        'model': RandomForestClassifier(class_weight='balanced', random_state=42),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [10, 20, None],
            'min_samples_split': [2, 5]
        }
    }
}


In [ ]:
# Train Traditional ML Models on All Magnifications
all_results = {}

for mag in ['40X', '100X', '200X', '400X']:
    X_train = all_features[mag]['X_train']
    y_train = all_features[mag]['y_train']
    X_test = all_features[mag]['X_test']
    y_test = all_features[mag]['y_test']

    mag_results = {}

    for name, config in models_params.items():
        grid_search = GridSearchCV(config['model'], config['params'], 
                                   cv=5, scoring=f1_scorer, n_jobs=-1)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)

        mag_results[name] = {
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1': f1_score(y_test, y_pred)
        }

    all_results[mag] = mag_results

In [ ]:
# CNN Architecture
class BreastCancerCNN(nn.Module):
    def __init__(self):
        super(BreastCancerCNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)

        self.fc1 = nn.Linear(128 * 28 * 28, 256)
        self.fc2 = nn.Linear(256, 2)

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 128 * 28 * 28)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [ ]:
# Train CNNs on All Magnifications
cnn_results = {}

for mag in ['40X', '100X', '200X', '400X']:
    train_loader = datasets_all_mags[mag]['train_loader']
    val_loader = datasets_all_mags[mag]['val_loader']
    test_loader = datasets_all_mags[mag]['test_loader']

    model = BreastCancerCNN().to(device)

    dataset = datasets_all_mags[mag]['dataset']
    benign_count = dataset.labels.count(0)
    malignant_count = dataset.labels.count(1)
    class_weights = torch.tensor([1.0/benign_count, 1.0/malignant_count]).to(device)
    class_weights = class_weights / class_weights.sum()

    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    best_val_f1 = 0
    best_model_state = None

    for epoch in range(20):
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        val_preds = []
        val_labels_list = []

        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                val_preds.extend(predicted.cpu().numpy())
                val_labels_list.extend(labels.numpy())

        val_f1 = f1_score(val_labels_list, val_preds)

        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            best_model_state = model.state_dict().copy()

    model.load_state_dict(best_model_state)

    model.eval()
    test_preds = []
    test_labels_list = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            test_preds.extend(predicted.cpu().numpy())
            test_labels_list.extend(labels.numpy())

    cnn_results[mag] = {
        'accuracy': accuracy_score(test_labels_list, test_preds),
        'precision': precision_score(test_labels_list, test_preds),
        'recall': recall_score(test_labels_list, test_preds),
        'f1': f1_score(test_labels_list, test_preds)
    }


In [ ]:
# Compile Results
all_data = {}

for mag in ['40X', '100X', '200X', '400X']:
    all_data[mag] = {}
    for model_name in ['Logistic Regression', 'Linear SVM', 'RBF SVM', 'Random Forest']:
        all_data[mag][model_name] = {
            'F1': all_results[mag][model_name]['f1'],
            'Accuracy': all_results[mag][model_name]['accuracy'],
            'Recall': all_results[mag][model_name]['recall'],
            'Precision': all_results[mag][model_name]['precision']
        }
    all_data[mag]['CNN'] = {
        'F1': cnn_results[mag]['f1'],
        'Accuracy': cnn_results[mag]['accuracy'],
        'Recall': cnn_results[mag]['recall'],
        'Precision': cnn_results[mag]['precision']
    }

results_data = []
for mag in ['40X', '100X', '200X', '400X']:
    for model in ['Logistic Regression', 'Linear SVM', 'RBF SVM', 'Random Forest', 'CNN']:
        results_data.append({
            'Magnification': mag,
            'Model': model,
            'F1': all_data[mag][model]['F1'],
            'Accuracy': all_data[mag][model]['Accuracy'],
            'Recall': all_data[mag][model]['Recall'],
            'Precision': all_data[mag][model]['Precision']
        })

results_df = pd.DataFrame(results_data)


In [ ]:
# Performance Comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

sns.barplot(data=results_df, x='Magnification', y='F1', hue='Model', ax=axes[0,0], palette='Set2')
axes[0,0].set_title('(a) F1 Score by Magnification', fontsize=12, fontweight='bold')
axes[0,0].set_ylim(0.8, 1.0)
axes[0,0].legend(fontsize=8, loc='lower right')

sns.barplot(data=results_df, x='Magnification', y='Accuracy', hue='Model', ax=axes[0,1], palette='Set2')
axes[0,1].set_title('(b) Accuracy by Magnification', fontsize=12, fontweight='bold')
axes[0,1].set_ylim(0.8, 1.0)
axes[0,1].legend(fontsize=8, loc='lower right')

sns.barplot(data=results_df, x='Magnification', y='Recall', hue='Model', ax=axes[1,0], palette='Set2')
axes[1,0].set_title('(c) Recall by Magnification', fontsize=12, fontweight='bold')
axes[1,0].set_ylim(0.8, 1.0)
axes[1,0].legend(fontsize=8, loc='lower right')

best_per_mag = results_df.loc[results_df.groupby('Magnification')['F1'].idxmax()]
axes[1,1].bar(best_per_mag['Magnification'], best_per_mag['F1'], 
              color=['#66c2a5', '#fc8d62', '#8da0cb', '#e78ac3'], edgecolor='black', linewidth=1.5)
axes[1,1].set_title('(d) Best Model per Magnification', fontsize=12, fontweight='bold')
axes[1,1].set_ylim(0.8, 1.0)
for i, (mag, f1, model) in enumerate(zip(best_per_mag['Magnification'], best_per_mag['F1'], best_per_mag['Model'])):
    axes[1,1].text(i, f1 + 0.005, f'{f1:.4f}\n{model}', ha='center', fontsize=7, fontweight='bold')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/figure2_performance.png', dpi=300, bbox_inches='tight')
plt.show()

# Confusion Matrix
cm = np.array([[82, 9], [0, 209]])

plt.figure(figsize=(7, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Benign', 'Malignant'],
            yticklabels=['Benign', 'Malignant'],
            annot_kws={'size': 14, 'weight': 'bold'},
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix: 100X RBF SVM', fontsize=14, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.text(2.6, 0.5, 'Accuracy: 97.14%\nPrecision: 96.05%\nRecall: 100.00%\nF1: 97.98%', 
         fontsize=10, bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/figure3_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:

# NOTE: During initial run, 200X data was accidentally identical to 400X due to 
# a data loading error. The issue has been fixed in Cells 6-12 above, but to avoid
# re-running the entire notebook (which takes 2+ hours), I reran everything for 200x, which is the code below


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, models
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
from tqdm import tqdm
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

base_path = '/content/drive/MyDrive/archive (2)/BreaKHis_v1/BreaKHis_v1/histology_slides/breast'

class BreastCancerDataset(Dataset):
    def __init__(self, base_path, magnification='400X', transform=None):
        self.base_path = base_path
        self.magnification = magnification
        self.transform = transform
        self.images = []
        self.labels = []
        
        for class_name, label in [('benign', 0), ('malignant', 1)]:
            class_path = os.path.join(base_path, class_name)
            
            for root, dirs, files in os.walk(class_path):
                if magnification in root:
                    for file in files:
                        if file.endswith('.png'):
                            img_path = os.path.join(root, file)
                            self.images.append(img_path)
                            self.labels.append(label)
        
        print(f"{magnification}: {len(self.images)} images ({self.labels.count(0)} benign, {self.labels.count(1)} malignant)")
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset_200X = BreastCancerDataset(base_path, magnification='200X', transform=transform)

In [ ]:
indices = list(range(len(dataset_200X)))
labels = dataset_200X.labels

train_idx, temp_idx = train_test_split(indices, test_size=0.30, random_state=42, stratify=labels)
temp_labels = [labels[i] for i in temp_idx]
val_idx, test_idx = train_test_split(temp_idx, test_size=0.50, random_state=42, stratify=temp_labels)

train_dataset = Subset(dataset_200X, train_idx)
val_dataset = Subset(dataset_200X, val_idx)
test_dataset = Subset(dataset_200X, test_idx)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

In [ ]:
resnet = models.resnet50(weights='IMAGENET1K_V1')
feature_extractor = nn.Sequential(*list(resnet.children())[:-1])
feature_extractor = feature_extractor.to(device)
feature_extractor.eval()

def extract_features(dataloader, extractor):
    features = []
    labels = []
    
    with torch.no_grad():
        for images, lbls in tqdm(dataloader):
            images = images.to(device)
            feats = extractor(images)
            feats = feats.view(feats.size(0), -1)
            features.append(feats.cpu().numpy())
            labels.append(lbls.numpy())
    
    features = np.vstack(features)
    labels = np.concatenate(labels)
    return features, labels

X_train_200X, y_train_200X = extract_features(train_loader, feature_extractor)
X_val_200X, y_val_200X = extract_features(val_loader, feature_extractor)
X_test_200X, y_test_200X = extract_features(test_loader, feature_extractor)

print(f"Train: {X_train_200X.shape}, Val: {X_val_200X.shape}, Test: {X_test_200X.shape}")

In [ ]:
# TRAIN ALL 200X MODELS
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
import time

f1_scorer = make_scorer(f1_score)

models_params = {
    'Logistic Regression': {
        'model': LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42),
        'params': {
            'C': [0.01, 0.1, 1, 10, 100],
            'penalty': ['l2']
        }
    },
    'Linear SVM': {
        'model': SVC(kernel='linear', class_weight='balanced', random_state=42),
        'params': {
            'C': [0.01, 0.1, 1, 10, 100]
        }
    },
    'RBF SVM': {
        'model': SVC(kernel='rbf', class_weight='balanced', random_state=42),
        'params': {
            'C': [0.1, 1, 10],
            'gamma': ['scale', 0.001, 0.01, 0.1]
        }
    },
    'Random Forest': {
        'model': RandomForestClassifier(class_weight='balanced', random_state=42),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [10, 20, None],
            'min_samples_split': [2, 5]
        }
    }
}

mag = '200X'
mag_results_200X = {}

for name, config in models_params.items():
    print(f"\n{name}...")
    start_time = time.time()
    
    grid_search = GridSearchCV(
        config['model'], 
        config['params'], 
        cv=5, 
        scoring=f1_scorer,
        n_jobs=-1
    )
    
    grid_search.fit(X_train_200X, y_train_200X)
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test_200X)
    
    mag_results_200X[name] = {
        'best_params': grid_search.best_params_,
        'accuracy': accuracy_score(y_test_200X, y_pred),
        'precision': precision_score(y_test_200X, y_pred),
        'recall': recall_score(y_test_200X, y_pred),
        'f1': f1_score(y_test_200X, y_pred),
        'train_time': time.time() - start_time
    }
    
    print(f"  F1: {mag_results_200X[name]['f1']:.4f}")
    print(f"  Accuracy: {mag_results_200X[name]['accuracy']:.4f}")
    print(f"  Recall: {mag_results_200X[name]['recall']:.4f}")
    print(f"  Time: {mag_results_200X[name]['train_time']:.1f}s")

for model_name in ['Logistic Regression', 'Linear SVM', 'RBF SVM', 'Random Forest']:
    res = mag_results_200X[model_name]
    print(f"{model_name:20s}: F1={res['f1']:.4f}, Acc={res['accuracy']:.4f}, Recall={res['recall']:.4f}")

In [ ]:
# TRAIN 200X CNN
import torch
import torch.nn as nn
import torch.optim as optim

class BreastCancerCNN(nn.Module):
    def __init__(self):
        super(BreastCancerCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(128 * 28 * 28, 256)
        self.fc2 = nn.Linear(256, 2)
        
    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))
        
        x = x.view(-1, 128 * 28 * 28)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

model = BreastCancerCNN().to(device)

benign_count = 623
malignant_count = 1390
class_weights = torch.tensor([1.0/benign_count, 1.0/malignant_count]).to(device)
class_weights = class_weights / class_weights.sum()

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20
best_val_f1 = 0
best_model_state = None

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    model.eval()
    val_preds = []
    val_labels_list = []
    
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            val_preds.extend(predicted.cpu().numpy())
            val_labels_list.extend(labels.numpy())
    
    val_f1 = f1_score(val_labels_list, val_preds)
    val_acc = accuracy_score(val_labels_list, val_preds)
    
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        best_model_state = model.state_dict().copy()
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {train_loss/len(train_loader):.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")

model.load_state_dict(best_model_state)

model.eval()
test_preds = []
test_labels_list = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_preds.extend(predicted.cpu().numpy())
        test_labels_list.extend(labels.numpy())

cnn_acc = accuracy_score(test_labels_list, test_preds)
cnn_prec = precision_score(test_labels_list, test_preds)
cnn_rec = recall_score(test_labels_list, test_preds)
cnn_f1 = f1_score(test_labels_list, test_preds)

print(f"\n200X CNN Test Results:")
print(f"  Accuracy:  {cnn_acc:.4f}")
print(f"  Precision: {cnn_prec:.4f}")
print(f"  Recall:    {cnn_rec:.4f}")
print(f"  F1 Score:  {cnn_f1:.4f}")